In [1]:
import numpy as np
import pandas as pd

In [2]:
import seaborn as sns
import matplotlib.pylab as plt

In [3]:
import torch
import torch.nn.functional as F
import pyro

In [4]:
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

In [5]:
from monteloanco import Model, Guide, GroupedBatchSampler, tmat_reshape, Template

### A deep state-space model for a consumer credit risk portfolio

This notebook outlines the development of a deep state-space model for consumer credit risk, built using [pyro.ai](https://pyro.ai/). At its core, the model employs Monte Carlo simulations for each loan, progressing through monthly timesteps. The hidden state at each step represents the loan’s status, with all accounts initially starting as current. From there, loans may transition to early payoff, arrears, or more commonly, remain current and advance to the next month.

The model requires 5 inputs: 
- `loan_amnt` the initial advance to the customer.
- `int_rate` the annual interest rate (as a percentage).
- `installment` the monthly payment according to the initial schedule.
- `total_pre_chargeoff` the total value of payments made against the account excluding recoveries.
- `num_timesteps` the number of months observed to date if training, or the desired length of the simulation.

The output used for validation is a simulation of hidden states (loan statuses) and payments, plus how those payments are attributed to principal and interest. Behind the scenes, the model also trains an embedding based on the loan account identifier, which effectively captures the performance characteristics of each specific loan. This embedding may serve several purposes, including:
- Simulating the performance of the existing portfolio.
- Extending the installment schedule to maturity to estimate the portfolio’s value if allowed to run off.
- Providing a low-dimensional representation of loan performance, enabling broader analysis beyond traditional good/bad account classifications for training applicant-level models.
- Reducing to a single risk dimension that represents the probability of default over any given time horizon.

We take a subset of the 2+ million accounts available here for speed.

In [6]:
df_train = pd.read_json('training.jsonl.gz', lines=True)
pd.testing.assert_index_equal(df_train.index, pd.RangeIndex(0, len(df_train)))

The model has been designed such that it can train / simulate a large number of accounts in parallel on a GPU. If you don't have a suitable GPU installed on your machine simply replace `cuda:0` here with `cpu`.

The data must be fed into the model in batches, where all sequences in a batch have the same length. We use the custom`GroupedBatchSampler` to define these batches.

In [7]:
device = 'cuda:0'
embedding_size = 3

In [8]:
batch_size = 100_000
dataset = df_train[['id', 'loan_amnt', 'int_rate', 'installment', 'n_report_d', 'total_pre_chargeoff']].to_dict(orient='records')
grouped_batch_sampler = GroupedBatchSampler(dataset, batch_size)

### Train the model

With the batches defined it's time to run the optimisation process, and tune the parameters. The loss here is the difference between the the total value of payments made on each account vs. those from the MC simulation.

In [9]:
%%time

# clear the param store in case we're in a REPL
pyro.clear_param_store()

# Initialize the model and guide
model = Model(embedding_size, device).to(device)
guide = Guide(device).to(device)

# Set up the optimizer and inference algorithm
optimizer = pyro.optim.Adam({"lr": 0.01})
svi = pyro.infer.SVI(model=model, guide=guide, optim=optimizer, loss=pyro.infer.Trace_ELBO())

# Run inference
#num_batches = grouped_batch_sampler.__len__()
num_iterations = 5_000
with tqdm(total=num_iterations, desc="Epochs", position=0) as epoch_pbar:
    for step in range(num_iterations):
    #with tqdm(total=num_batches, desc=f"Epoch {step + 1}", position=1, leave=False) as batch_pbar:
        losses = []
        for batch_id, batch in enumerate(DataLoader(dataset, batch_sampler=grouped_batch_sampler, num_workers=1)):
            losses.append(svi.step(
                batch_id=batch_id,
                batch_idx=torch.arange(len(batch['id'])).to(device), 
                installments=batch['installment'].to(device), 
                loan_amnt=batch['loan_amnt'].to(device), 
                int_rate=batch['int_rate'].to(device),
                total_pre_chargeoff=batch['total_pre_chargeoff'].to(device),
                num_timesteps=batch['n_report_d'].unique().item()))
            #batch_pbar.update(1)
        if step % np.ceil(num_iterations/100) == 0:
            print(f"Step {step} : Loss = {np.sum(losses)}")
        epoch_pbar.update(1)


Epochs:   0%|          | 0/5000 [00:00<?, ?it/s]

Step 0 : Loss = 238226234.559069
Step 50 : Loss = 192794185.52017316
Step 100 : Loss = 163497940.21337074
Step 150 : Loss = 138727092.13574007
Step 200 : Loss = 113021285.73833816
Step 250 : Loss = 91080740.38268556
Step 300 : Loss = 71066493.25068812
Step 350 : Loss = 60264069.054535754
Step 400 : Loss = 51705178.67244413
Step 450 : Loss = 44873204.83061723
Step 500 : Loss = 39630970.4511991
Step 550 : Loss = 37853536.682191536
Step 600 : Loss = 34543694.156989686
Step 650 : Loss = 31750359.110879995
Step 700 : Loss = 30902332.196030255
Step 750 : Loss = 28614311.146266293
Step 800 : Loss = 27841487.79360258
Step 850 : Loss = 25944994.71903947
Step 900 : Loss = 24860335.29281549
Step 950 : Loss = 24669130.516366906
Step 1000 : Loss = 24364775.612344656
Step 1050 : Loss = 24325403.20592795
Step 1100 : Loss = 23043025.695626803
Step 1150 : Loss = 22478891.269797314
Step 1200 : Loss = 22505432.306996763
Step 1250 : Loss = 21453355.320758153
Step 1300 : Loss = 20975146.57255727
Step 1350 

### Save the model

Save model parameters to a file for inference in another notebook.

In [10]:
model.to(device)

Model()

In [11]:
pyro.get_param_store().save('param_store.pt')
torch.save(model.state_dict(), 'model_params.pt')
torch.save(guide.state_dict(), "guide_params.pt")